### **Import Library**

In [1]:
import tensorflow as tf

In [2]:
batch_size = 16
EPOCHS = 100
WARMUP_EPOCHS = 2
LEARNING_RATE = 1e-4
WARMUP_LEARNING_RATE = 1e-3
HEIGHT = 224
WIDTH = 224
CANAL = 3
N_CLASSES = 4
ES_PATIENCE = 5
RLROP_PATIENCE = 3
DECAY_DROP = 0.5

### **EVALUASI DENSNET**

In [3]:
input_shape = (224,224,3)
model_input =tf.keras.Input(shape=input_shape)

Base_model1 =tf.keras.applications.DenseNet201(input_shape=input_shape, input_tensor=model_input, include_top=False, weights=None)
for layer in Base_model1.layers:
    layer.trainable = True

In [4]:
Base_model1_last_layer = Base_model1.get_layer('relu')
print('last layer output shape:',Base_model1_last_layer.output_shape)
Base_model1_last_output = Base_model1_last_layer.output

last layer output shape: (None, 7, 7, 1920)


In [5]:
x1 =tf.keras.layers.GlobalAveragePooling2D()(Base_model1_last_output)
x1 =tf.keras.layers.Dropout(0.25)(x1)
x1 =tf.keras.layers.Dense(512, activation='relu')(x1)
x1 =tf.keras.layers.Dropout(0.25)(x1)
final_output1 =tf.keras.layers.Dense(4, activation='softmax', name='final_output')(x1)
DensNet201_model =tf.keras.models.Model(model_input, final_output1)
metric_list = ["accuracy"]
optimizer =tf.keras.optimizers.Adam(lr=5.0000e-05)
DensNet201_model.compile(optimizer=optimizer, loss="categorical_crossentropy",  metrics=metric_list)
#DensNet201_model.load_weights('/content/drive/My Drive/Colab Notebooks/DATA RD/MODEL/Weights/K=3/Weight_DensNet201_Optimal_(k=3).h5')

### **EVALUASI INCEPTIONV3**

In [6]:
Base_model2 =tf.keras.applications.InceptionV3(input_shape=input_shape, input_tensor=model_input, include_top=False, weights=None)
for layer in Base_model2.layers:
    layer.trainable = True

In [7]:
Base_model2_last_layer = Base_model2.get_layer('mixed10')
print('last layer output shape:', Base_model2_last_layer.output_shape)
Base_model2_last_output = Base_model2_last_layer.output

last layer output shape: (None, 5, 5, 2048)


In [9]:
x2 =tf.keras.layers.GlobalAveragePooling2D()(Base_model2_last_output)
x2 =tf.keras.layers.Dropout(0.25)(x2)
x2 =tf.keras.layers.Dense(1024, activation='relu')(x2)
x2 =tf.keras.layers.Dropout(0.25)(x2)
final_output2 =tf.keras.layers.Dense(4, activation='softmax', name='final_output2')(x2)
InceptionV3_model =tf.keras.models.Model(model_input, final_output2)
metric_list = ["accuracy"]
optimizer = tf.keras.optimizers.Adam( 6.2500e-06)
InceptionV3_model.compile(optimizer=optimizer, loss="categorical_crossentropy",  metrics=metric_list)
#InceptionV3_model.load_weights('/content/drive/My Drive/Colab Notebooks/DATA RD/MODEL/Weights/K=3/Weight_InceptionV3_Optimal_(k=3).h5')

### **EVALUASI MOBILENETV3**

In [10]:
Base_model3 =tf.keras.applications.MobileNetV2(input_shape=input_shape, input_tensor=model_input, include_top=False, weights=None)
for layer in Base_model3.layers:
    layer.trainable = True

In [11]:
Base_model3_last_layer = Base_model3.get_layer('out_relu')
print('last layer output shape:', Base_model3_last_layer.output_shape)
Base_model3_last_output = Base_model3_last_layer.output

last layer output shape: (None, 7, 7, 1280)


In [12]:
x3 =tf.keras.layers.GlobalAveragePooling2D()(Base_model3_last_output)
x3 =tf.keras.layers.Dropout(0.5)(x3)
x3 =tf.keras.layers.Dense(512, activation='relu')(x3)
x3 =tf.keras.layers.Dropout(0.5)(x3)
final_output3 =tf.keras.layers.Dense(4, activation='softmax', name='final_output3')(x3)
MobileNetV2_model =tf.keras.models.Model(model_input, final_output3)
metric_list = ["accuracy"]
optimizer = tf.keras.optimizers.Adam(lr=2.5000e-05)
MobileNetV2_model.compile(optimizer=optimizer, loss="categorical_crossentropy",  metrics=metric_list)
#MobileNetV2_model.load_weights('/content/drive/My Drive/Colab Notebooks/DATA RD/MODEL/Weights/K=3/Weight_MobileNetV2_Optimal_(3).h5')

In [13]:
Base_model4 =tf.keras.applications.EfficientNetB7(input_shape=input_shape, input_tensor=model_input, include_top=False, weights=None)
for layer in Base_model4.layers:
    layer.trainable = True

In [15]:
Base_model4_last_layer = Base_model4.get_layer('top_activation')
print('last layer output shape:', Base_model4_last_layer.output_shape)
Base_model4_last_output = Base_model4_last_layer.output

last layer output shape: (None, 7, 7, 2560)


In [16]:
x4 =tf.keras.layers.GlobalAveragePooling2D()(Base_model4_last_output)
x4 =tf.keras.layers.Dropout(0.5)(x4)
x4 =tf.keras.layers.Dense(512, activation='relu')(x3)
x4 =tf.keras.layers.Dropout(0.5)(x4)
final_output4 =tf.keras.layers.Dense(4, activation='softmax', name='final_output3')(x3)
EfficientNetB7_model =tf.keras.models.Model(model_input, final_output4)
metric_list = ["accuracy"]
optimizer = tf.keras.optimizers.Adam(lr=2.5000e-05)
EfficientNetB7_model.compile(optimizer=optimizer, loss="categorical_crossentropy",  metrics=metric_list)

### **ENSAMBLE**

In [17]:
def ensemble(models, model_input):
    outputs = [model.outputs[0] for model in models]
    y =tf.keras.layers.Average()(outputs)
    model =tf.keras.Model(model_input,y,name='ensemble')
    return model

In [18]:
ensemble_model = ensemble([DensNet201_model,InceptionV3_model,EfficientNetB7_model], model_input)
metric_list = ["accuracy"]
optimizer =tf.keras.optimizers.Adam(lr=2.5000e-05)
ensemble_model.compile(optimizer=optimizer, loss="categorical_crossentropy",  metrics=metric_list)